<a href="https://colab.research.google.com/github/elinoyam/FindMyParking/blob/WorkingWithColab/CalculateProbabilitiesFromGeojson.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import json
import requests
import numpy as np
!pip install geojson
import geojson

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
def find_osm_node_id(coordinate):
  lat = coordinate[0]
  lon = coordinate[1]
  # get the way id - request json format of the reverse method of OSM librery nominatim
  request_url = "https://nominatim.openstreetmap.org/reverse?format=geojson&osm_type=N&lat=" + f'{lat:f}' + "&lon=" + f'{lon:f}'
  #print(request_url)
  response = requests.get(request_url)
  way_id_response = json.loads(response.text)
  #print(way_id_response)
  osm_type = way_id_response['features'][0]['properties']['osm_type']
  node_id = way_id_response['features'][0]['properties']['osm_id'] 
  if osm_type != "node":
    print("not node:")
    print(node_id)

  return node_id

In [4]:
def calculate_coords(list_of_old_coorinates):
  transformed_coordinates = []
  for old_cord in list_of_old_coorinates:
    old_lat = old_cord[0]
    old_lon = old_cord[1]
    # get the latitude and longitude in the right coordinates 
    # from the original format EPSG:25833 to format: EPSG:4326
    request_url = "https://epsg.io/trans?s_srs=25833&t_srs=4326&x="+str(old_lat)+"&y=" + str(old_lon)
    response = requests.get(request_url)
    transform_coordinates_response = json.loads(response.text)
    trans_lon = transform_coordinates_response['x']
    trans_lat = transform_coordinates_response['y']
    transformed_coordinates.append([float(trans_lat), float(trans_lon)])
  
  return transformed_coordinates


In [11]:
def transform_line(line):
  old_coords = line['geometry']['coordinates'][0][0]
  transformed_coords = calculate_coords(old_coords)
  osm_id = find_osm_node_id(transformed_coords[0])
  # calculate parking area
  all_lats = [x for x,y in transformed_coords]
  all_lons = [y for x,y in transformed_coords]
  parking_area = polygon_area(all_lats, all_lons) # we multiply by 1000 to convet to m2 instead of km2
  # calculate number of parking spots in the parking area
  single_parking_area = 4.88 * 2.44
  number_of_parking = parking_area / single_parking_area
  # print(number_of_parking)
  # get the label (parking empty/occupied)
  line_label = line['properties']['label_x']

  return osm_id, line_label, number_of_parking

In [ ]:
# line = '{ "type": "Feature", "properties": { "id": 1, "label_x": "PK-space-occupied", "True_or_False": null, "label_y": null, "nearest_geometry_dis": null, "parking_area_m2": null, "layer": "Annotaions Predictions", "path": "F:/Berlin-DOP_2020/Predictions/Previously created maps/Annotations/Annotaions.geojson" }, "geometry": { "type": "MultiPolygon", "coordinates": [ [ [ [ 391008.0878, 5821088.3568 ], [ 391002.407, 5821095.493 ], [ 391004.3319, 5821096.9484 ], [ 391009.8718, 5821089.4366 ], [ 391008.1817, 5821088.2629 ], [ 391008.0878, 5821088.3568 ] ] ] ] } }'
# #line = line.replace("null", "None")
# json_line = json.loads(line)
# transform_line(json_line)

In [ ]:
# osm_id = find_osm_node_id([52.54203249, 13.41329482])
# print(osm_id)

692194370


In [ ]:
# input_coord = [ [ 392395.9, 5822510.25 ], [ 392396.65, 5822510.8 ], [ 392398.25, 5822513.5 ], [ 392398.15, 5822513.8 ], [ 392397.6, 5822514.35 ], [ 392397.1, 5822514.65 ], [ 392396.6, 5822514.55 ], [ 392396.25, 5822514.2 ], [ 392395.15, 5822512.6 ], [ 392394.95, 5822512.1 ], [ 392394.95, 5822511.6 ], [ 392395.35, 5822510.7 ], [ 392395.9, 5822510.25 ]]

# output_coord = calculate_coords(input_coord)

# print(output_coord)

# for cord in output_coord:
#   found_id = find_osm_node_id(cord)
#   print(found_id)

In [7]:
# first way to calculate area on the earth surface

!pip install area
from area import area
poly_obj = {'type':'Polygon','coordinates':[[[52.54203249, 13.41329482], [52.54203758, 13.41330569], [52.54206216, 13.4133284], [52.54206483, 13.41332683], [52.54206967, 13.41331855], [52.54207226, 13.41331108], [52.54207127, 13.41330374], [52.54206805, 13.4132987], [52.54205346, 13.413283], [52.54204892, 13.41328021], [52.54204443, 13.41328038], [52.54203642, 13.41328656], [52.54203249, 13.41329482]]]}
area(poly_obj)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for area: filename=area-1.1.1-py3-none-any.whl size=3626 sha256=461e31f198000aaf9e36bd7881acc5412541328e6c2f20751cec56663c8a5473
  Stored in directory: /root/.cache/pip/wheels/59/e8/a2/0a9be8239358f5e3b48e1b5cb756f98b0f992b881e1b8b21b2
Successfully built area


13.324528739947908

In [8]:
# second (and more precise) way for calculating the area on earth surface

def polygon_area(lats, lons, radius = 6378137):
    """
    Computes area of spherical polygon, assuming spherical Earth. 
    Returns result in ratio of the sphere's area if the radius is specified.
    Otherwise, in the units of provided radius.
    lats and lons are in degrees.
    """
    from numpy import arctan2, cos, sin, sqrt, pi, power, append, diff, deg2rad
    lats = np.deg2rad(lats)
    lons = np.deg2rad(lons)

    # Line integral based on Green's Theorem, assumes spherical Earth

    #close polygon
    if lats[0]!=lats[-1]:
        lats = append(lats, lats[0])
        lons = append(lons, lons[0])

    #colatitudes relative to (0,0)
    a = sin(lats/2)**2 + cos(lats)* sin(lons/2)**2
    colat = 2*arctan2( sqrt(a), sqrt(1-a) )

    #azimuths relative to (0,0)
    az = arctan2(cos(lats) * sin(lons), sin(lats)) % (2*pi)

    # Calculate diffs
    # daz = diff(az) % (2*pi)
    daz = diff(az)
    daz = (daz + pi) % (2 * pi) - pi

    deltas=diff(colat)/2
    colat=colat[0:-1]+deltas

    # Perform integral
    integrands = (1-cos(colat)) * daz

    # Integrate 
    area = abs(sum(integrands))/(4*pi)

    area = min(area,1-area)
    if radius is not None: #return in units of radius
        return area * 4*pi*radius**2
    else: #return in ratio of sphere total area
        return area

In [12]:
file_path = "parking.geojson"
# count the parking spots members:
empty_pk_spots = {}
occupied_pk_spots = {}

# open the file using geojson (less updated)
with open(file_path) as f:
    data_file = geojson.load(f)
    features_array = data_file['features']

    for feature in features_array:
      node_id, label, amount = transform_line(feature)
      if label == 'PK-space-occupied':
        occupied_count = occupied_pk_spots.setdefault(node_id, 0)
        occupied_pk_spots[node_id] = occupied_count + 1
      elif label == 'PK-space-empty':
        empty_count = empty_pk_spots.setdefault(node_id, 0)
        empty_pk_spots[node_id] = empty_count + 1 


not node:
5758790
not node:
5758790
not node:
390623189
not node:
390623189
not node:
390623189
not node:
390623189
not node:
53332487
not node:
147659855
not node:
14375945
not node:
14375945
not node:
14375945
not node:
14375945
not node:
116676564
not node:
116676564
not node:
116676564
not node:
116676564
not node:
116676564
not node:
116676564
not node:
180662834
not node:
180662836
not node:
180662834
not node:
180662834
not node:
381196085
not node:
391005378
not node:
267229894
not node:
267229891
not node:
8096771
not node:
105786490
not node:
105786498
not node:
105786498
not node:
201198297
not node:
105786501
not node:
201198297
not node:
105785524
not node:
105785524
not node:
105785524
not node:
105785524
not node:
201198297
not node:
105786501
not node:
105785522
not node:
105785522
not node:
105785522
not node:
116676564
not node:
116676564
not node:
116676564
not node:
116676564
not node:
8003639
not node:
8003639
not node:
301595370
not node:
301595370
not node:
30159

JSONDecodeError: ignored

In [ ]:
def get_prob_for_node(node_id, index):
    # print('index = ' + str(index))
    # print('node_id = ' + str(node_id))
    f = open("probabilities.osm", "r")
    flag=-1
    for line in f.readlines():
        if line.startswith('\t\t<nd ref'):
            start_index = (line.index('=')) + 2 #the starting of the node_id number
            end_index= (line.index('/')) -1            
            if (line[start_index:end_index]) == node_id:
              flag=0  # it means that the next tag we will see we will go inside
        elif flag == 0 and line.startswith('\t\t<tag'):
          start = line.index('v')
          end = len(line) - 4
          temp_list = line[start+3:end].split(',')
          prob_list = [eval(i) for i in temp_list]
          print('prob_list = ' + str(prob_list))
          print('prob_list value = ' + str(prob_list[index]))
          return prob_list[index]
    return 0 #if didn't find- return 0

In [ ]:
def most_common_highest_probability():
    file = open("probabilities.osm", "r")
    bucket_list = [0] * 168 # 24 * 7
    for line in file.readlines():
        if line.startswith('\t\t<tag'):
            index = line.index('v')
            end = len(line) - line[::-1].index('"') -1
            temp_list = line[index+3:end].split(',')
            for index in range(len(temp_list)):
                if temp_list[index] not in ['', '0']:
                    bucket_list[index] += 1
    print('bucket_list = ' + str(bucket_list))
    return bucket_list.index(max(bucket_list))

In [ ]:
def prepare_data_for_model():
  data_list=[]
  relevant_index= most_common_highest_probability()
  for node_id in empty_pk_spots.keys():
    num_of_empty= empty_pk_spots[node_id]
    if node_id in occupied_pk_spots:
      num_of_occupied= occupied_pk_spots[node_id]
    else:
      num_of_occupied=0
    prob_for_empty_parking= get_prob_for_node(node_id, relevant_index)
    data_list.append([num_of_empty, num_of_occupied, prob_for_empty_parking])

  for node_id in occupied_pk_spots.keys():
    if node_id not in empty_pk_spots: # so we didn't go through it yet and the probability is 0
      prob_for_empty_parking= get_prob_for_node(node_id, relevant_index)
      data_list.append([0, occupied_pk_spots[node_id], prob_for_empty_parking])
  
  return data_list


In [ ]:
data = prepare_data_for_model()
print(data)